# EXAMPLE LOOP THROUGH SCENARIO COMBINATIONS FOR ONE MODEL

*using classes and calibration methods*

In [ ]:
import logging
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from CalibrationFunctions import DataPreprocessor, Calibrator, ModelEvaluator

In [ ]:

logging.basicConfig(level=logging.INFO)

In [ ]:
# ===========================================
# INIT LISTS & CONSTANTS
# ===========================================
results = []
dataset_sizes    = [10_000, 50_000, 380_000]
imbalance_ratios = [0.004, 0.05, 0.2]
calibration_methods = [
    'isotonic', 'sigmoid', 'exponential', 'polynomial',
    'bbq', 'venn_abers', 'hist_binning'
]

# ===========================================
# LOOP THROUGH SCENARIOS
# ===========================================
for size in dataset_sizes:
    # Load + sample + split
    bp = DataPreprocessor('INSERT')
    bp.load_data('my_path')
    bp.sample_dataset(sample_size=size)
    bp.prepare_data(test_size=0.2, calib_size=0.25)

    for ratio in imbalance_ratios:
        # Balance training set to desired positive:negative ratio
        train_df = pd.concat([bp.X_train, bp.y_train], axis=1)
        bp_balance = DataPreprocessor('INSERT')
        bp_balance.train_df = train_df.copy()
        bp_balance.X_train = train_df.drop('INSERT', axis=1)
        bp_balance.y_train = train_df['INSERT']
        bp_balance.balance_target_variable(method='ratio', target_ratio=ratio)
        X_train_bal, y_train_bal = bp_balance.X_train, bp_balance.y_train

        for use_weight in (False, True):
            
# ======================================================
# compute dynamic class weight for additional comparison (unweighted VS weighted)
# =======================================================
            if use_weight:
                spw = (y_train_bal == 0).sum() / (y_train_bal == 1).sum()
                class_weight = {0: 1, 1: spw}
            else:
                class_weight = None

            # initialize ONE model (example)
            model = RandomForestClassifier(
                n_estimators   = 500,
                max_depth      = 6,
                class_weight   = class_weight,
                random_state   = 243,
                n_jobs         = -1,
                oob_score      = False
            )
            model.fit(X_train_bal, y_train_bal)

            # calibration evaluation
            calib = Calibrator(
                model     = model,
                calib_set = (bp.X_calib, bp.y_calib),
                test_set  = (bp.X_test, bp.y_test)
            )
            calib.evaluate('uncalibrated')
            u_ece     = calib.metrics['uncalibrated']['ece']
            u_logloss = calib.metrics['uncalibrated']['log_loss']

            # apply calibration methods
            for m in calibration_methods:
                try:
                    if m in ('isotonic', 'sigmoid'):
                        calib.fit_calibrator(m, cv=5)
                    elif m == 'hist_binning':
                        calib.fit_calibrator(m, n_bins=15)
                    elif m == 'bbq':
                        calib.fit_calibrator(m, n_bins=15, prior=0.5, max_samples=300_000)
                    elif m == 'exponential':
                        calib.fit_calibrator(m)
                    elif m == 'polynomial':
                        calib.fit_calibrator(m, degree=3)
                    elif m == 'venn_abers':
                        calib.fit_calibrator(m, cal_size=0.2, random_state=101)
                    else:
                        raise ValueError(f"Unknown calibration method `{m}`")
                    calib.evaluate(m)
                except Exception:
                    logging.warning(
                        f"Calibration {m} failed at size={size}, ratio={ratio}, weight={use_weight}"
                    )

            # select best calibration by weighted score
            best_score, best_method = float('inf'), None
            for method, mets in calib.metrics.items():
                if method == 'uncalibrated':
                    continue
                score = 0.7 * mets['ece'] + 0.3 * mets['log_loss']
                if score < best_score:
                    best_score, best_method = score, method

            # compute pre- and post-calibration best scores
            pre_score  = 0.7 * u_ece + 0.3 * u_logloss
            post_score = best_score if best_method is not None else pre_score
            pct_red    = 0.0 if pre_score == 0 else 100 * (pre_score - post_score) / pre_score

            # record results
            results.append({
                'MODEL'           : 'RANDOM_FOREST',
                'SIZE'            : size,
                'RATIO'           : ratio,
                'MIN-COUNT'       : int((y_train_bal == 1).sum()),
                'WEIGHT'          : use_weight,
                'PRE-ECE'         : u_ece,
                'PRE-logloss'     : u_logloss,
                'PRE-BEST-SCORE'  : pre_score,
                'BEST-CAL'        : best_method or 'none',
                'POST-ECE'        : calib.metrics.get(best_method, {}).get('ece', u_ece),
                'POST-logloss'    : calib.metrics.get(best_method, {}).get('log_loss', u_logloss),
                'BEST-SCORE'      : post_score,
                'VARIATION'       : pct_red
            })

# save results to CSV
results_df = pd.DataFrame(results)
results_df.to_csv('rf_bench.csv', index=False)
logging.info("RF benchmark complete, results saved to rf_bench.csv")


# We can think of displaying the best method selected in the table 
